# Variational Quantum Eigensolver



In [1]:
import cudaq

geometry = [('H', (0., 0., 0.)), ('H', (0., 0., .7474))]

In [2]:
molecule, data = cudaq.chemistry.create_molecular_hamiltonian(geometry, 'sto-3g', 1, 0)

/packages/envs/CUDA-Quantum-0.5.0/lib/python3.10/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [3]:
electron_count = data.n_electrons
qubit_count = 2 * data.n_orbitals

kernel, angles = cudaq.make_kernel(list)
qubits = kernel.qalloc(qubit_count)

# Prepare the Hartree Fock State.
kernel.x(qubits[0])
kernel.x(qubits[1])

# Adds parameterized gates based on the UCCSD ansatz.
cudaq.kernels.uccsd(kernel, qubits, angles, electron_count, qubit_count)

parameter_count = cudaq.kernels.uccsd_num_parameters(electron_count,
                                                     qubit_count)

optimizer = cudaq.optimizers.COBYLA()

energy, parameters = cudaq.vqe(kernel,
                               molecule,
                               optimizer,
                               parameter_count=parameter_count)

print(energy)

-1.137176147940608


We see that the energy has converged to the ground state minimum as expected for $H_{2}$.